In [1]:
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━

# Images per domain# 

In [2]:
import os

root_dir = "/kaggle/input/pacs-dataset/kfold"
domains = ["photo", "art_painting", "cartoon", "sketch"]

for domain in domains:
    domain_path = os.path.join(root_dir, domain)
    count = 0
    for cls in os.listdir(domain_path):
        cls_path = os.path.join(domain_path, cls)
        count += len(os.listdir(cls_path))
    print(f"{domain} has {count} images")


photo has 1670 images
art_painting has 2048 images
cartoon has 2344 images
sketch has 3929 images


**Clip domain generalization using patience learning(test on cartoon)**

In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"


clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
clip_model.eval().to(device)
for param in clip_model.parameters():
    param.requires_grad = False  # Freeze CLIP weights


class CLIPMLPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super().__init__()
        self.clip = clip_model
        self.mlp = nn.Sequential(
            nn.Linear(self.clip.visual.output_dim, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        with torch.no_grad():
            feats = self.clip.encode_image(x)
        return self.mlp(feats)


def get_loader(domains, root, batch_size=32, shuffle=False):
    dataset = []
    for domain in domains:
        ds = datasets.ImageFolder(os.path.join(root, domain), transform=preprocess)
        dataset.extend(ds.samples)
    base_ds = datasets.ImageFolder(os.path.join(root, domains[0]), transform=preprocess)
    base_ds.samples = dataset
    loader = DataLoader(base_ds, batch_size=batch_size, shuffle=shuffle)
    return loader, len(base_ds.classes)

PACS_PATH = "/kaggle/input/pacs-dataset/kfold"
train_domains = ["photo", "art_painting", "sketch"]
test_domain = "cartoon"

train_loader, num_classes = get_loader(train_domains, root=PACS_PATH, shuffle=True)
test_loader, _ = get_loader([test_domain], root=PACS_PATH, shuffle=False)


model = CLIPMLPClassifier(clip_model, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.mlp.parameters(), lr=1e-4)

# Early stopping 
best_acc = 0
wait = 0
patience = 10
best_model_state = None


for epoch in range(50): 
    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(imgs)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} - Training Loss: {total_loss:.4f}")

   
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = 100 * correct / total
    print(f"Test Accuracy on '{test_domain}': {acc:.2f}%")

    # Early stopping check
    if acc > best_acc:
        best_acc = acc
        best_model_state = model.state_dict()
        wait = 0
        print("Accuracy improved. Model saved.")
    else:
        wait += 1
        print(f"No improvement. Wait count: {wait}/{patience}")
        if wait >= patience:
            print("Early stopping triggered.")
            break


if best_model_state:
    model.load_state_dict(best_model_state)
    print(f"Best model with {best_acc:.2f}% accuracy loaded.")


Epoch 1 - Training: 100%|██████████| 239/239 [00:39<00:00,  6.04it/s]


Epoch 1 - Training Loss: 211.7789
Test Accuracy on 'cartoon': 98.51%
Accuracy improved. Model saved.


Epoch 2 - Training: 100%|██████████| 239/239 [00:39<00:00,  6.10it/s]


Epoch 2 - Training Loss: 43.3481
Test Accuracy on 'cartoon': 98.46%
No improvement. Wait count: 1/10


Epoch 3 - Training: 100%|██████████| 239/239 [00:38<00:00,  6.25it/s]


...

Epoch 12 - Training Loss: 14.3917
Test Accuracy on 'cartoon': 98.42%
No improvement. Wait count: 8/10


Epoch 13 - Training: 100%|██████████| 239/239 [00:39<00:00,  6.06it/s]


Epoch 13 - Training Loss: 13.7378
Test Accuracy on 'cartoon': 98.12%
No improvement. Wait count: 9/10


Epoch 14 - Training: 100%|██████████| 239/239 [00:39<00:00,  6.09it/s]


Epoch 14 - Training Loss: 13.1868
Test Accuracy on 'cartoon': 98.12%
No improvement. Wait count: 10/10
Early stopping triggered.
Best model with 98.72% accuracy loaded.


**Clip domain generalization using patience learning(test on art_painting)**

In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"


clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
clip_model.eval().to(device)
for param in clip_model.parameters():
    param.requires_grad = False 


class CLIPMLPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super().__init__()
        self.clip = clip_model
        self.mlp = nn.Sequential(
            nn.Linear(self.clip.visual.output_dim, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        with torch.no_grad():
            feats = self.clip.encode_image(x)
        return self.mlp(feats)


def get_loader(domains, root, batch_size=32, shuffle=False):
    dataset = []
    for domain in domains:
        ds = datasets.ImageFolder(os.path.join(root, domain), transform=preprocess)
        dataset.extend(ds.samples)
    base_ds = datasets.ImageFolder(os.path.join(root, domains[0]), transform=preprocess)
    base_ds.samples = dataset
    loader = DataLoader(base_ds, batch_size=batch_size, shuffle=shuffle)
    return loader, len(base_ds.classes)

PACS_PATH = "/kaggle/input/pacs-dataset/kfold"
train_domains = ["photo", "cartoon", "sketch"]
test_domain = "art_painting"

train_loader, num_classes = get_loader(train_domains, root=PACS_PATH, shuffle=True)
test_loader, _ = get_loader([test_domain], root=PACS_PATH, shuffle=False)


model = CLIPMLPClassifier(clip_model, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.mlp.parameters(), lr=1e-4)

# Early stopping 
best_acc = 0
wait = 0
patience = 10
best_model_state = None


for epoch in range(50): 
    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(imgs)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} - Training Loss: {total_loss:.4f}")

   
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = 100 * correct / total
    print(f"Test Accuracy on '{test_domain}': {acc:.2f}%")

    # Early stopping check
    if acc > best_acc:
        best_acc = acc
        best_model_state = model.state_dict()
        wait = 0
        print("Accuracy improved. Model saved.")
    else:
        wait += 1
        print(f"No improvement. Wait count: {wait}/{patience}")
        if wait >= patience:
            print("Early stopping triggered.")
            break


if best_model_state:
    model.load_state_dict(best_model_state)
    print(f"Best model with {best_acc:.2f}% accuracy loaded.")


Epoch 1 - Training: 100%|██████████| 249/249 [00:41<00:00,  5.95it/s]


Epoch 1 - Training Loss: 201.9480
Test Accuracy on 'art_painting': 97.31%
Accuracy improved. Model saved.


Epoch 2 - Training: 100%|██████████| 249/249 [00:40<00:00,  6.15it/s]


Epoch 2 - Training Loss: 38.7605
Test Accuracy on 'art_painting': 97.51%
Accuracy improved. Model saved.


Epoch 3 - Training: 100%|██████████| 249/249 [00:40<00:00,  6.15it/s]


...

Epoch 12 - Training Loss: 13.5028
Test Accuracy on 'art_painting': 97.07%
No improvement. Wait count: 8/10


Epoch 13 - Training: 100%|██████████| 249/249 [00:41<00:00,  6.03it/s]


Epoch 13 - Training Loss: 12.9832
Test Accuracy on 'art_painting': 96.92%
No improvement. Wait count: 9/10


Epoch 14 - Training: 100%|██████████| 249/249 [00:40<00:00,  6.20it/s]


Epoch 14 - Training Loss: 12.8201
Test Accuracy on 'art_painting': 96.83%
No improvement. Wait count: 10/10
Early stopping triggered.
Best model with 97.71% accuracy loaded.


**Domain Generalization using CLIP(test on cartoon**)

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"


clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
clip_model.eval().to(device)
for param in clip_model.parameters():
    param.requires_grad = False  # freeze CLIP

#  classifier  (512 → 256 → 7)
class CLIPMLPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super().__init__()
        self.clip = clip_model
        self.mlp = nn.Sequential(
            nn.Linear(self.clip.visual.output_dim, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        with torch.no_grad():
            feats = self.clip.encode_image(x)
        return self.mlp(feats)


def get_loader(domains, root, batch_size=32, shuffle=False):
    dataset = []
    for domain in domains:
        ds = datasets.ImageFolder(os.path.join(root, domain), transform=preprocess)
        dataset.extend(ds.samples)
    base_ds = datasets.ImageFolder(os.path.join(root, domains[0]), transform=preprocess)
    base_ds.samples = dataset
    loader = DataLoader(base_ds, batch_size=batch_size, shuffle=shuffle)
    return loader, len(base_ds.classes)


PACS_PATH = "/kaggle/input/pacs-dataset/kfold"
train_domains = ["photo", "art_painting", "sketch"]
test_domain = "cartoon"

train_loader, num_classes = get_loader(train_domains, root=PACS_PATH, shuffle=True)
test_loader, _ = get_loader([test_domain], root=PACS_PATH, shuffle=False)


model = CLIPMLPClassifier(clip_model, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.mlp.parameters(), lr=1e-4)

# Train + evaluate
for epoch in range(20):
    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} [Training]"):
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(imgs)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"\nEpoch {epoch+1} Training Loss: {total_loss:.4f}")

    
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = 100 * correct / total
    print(f"Test Accuracy on '{test_domain}': {acc:.2f}%\n")


Epoch 1 [Training]: 100%|██████████| 239/239 [00:39<00:00,  6.02it/s]



Epoch 1 Training Loss: 212.3339
Test Accuracy on 'cartoon': 98.46%



Epoch 2 [Training]: 100%|██████████| 239/239 [00:39<00:00,  5.99it/s]



Epoch 2 Training Loss: 42.9130
Test Accuracy on 'cartoon': 98.63%



Epoch 3 [Training]: 100%|██████████| 239/239 [00:39<00:00,  6.09it/s]


...


Epoch 18 Training Loss: 11.6934
Test Accuracy on 'cartoon': 97.95%



Epoch 19 [Training]: 100%|██████████| 239/239 [00:39<00:00,  6.07it/s]



Epoch 19 Training Loss: 11.0945
Test Accuracy on 'cartoon': 98.12%



Epoch 20 [Training]: 100%|██████████| 239/239 [00:39<00:00,  6.08it/s]



Epoch 20 Training Loss: 10.7250
Test Accuracy on 'cartoon': 98.51%



**Domain Generalization using CLIP(on art_painting**)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"


clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
clip_model.eval().to(device)
for param in clip_model.parameters():
    param.requires_grad = False  

# classifier  (512 → 256 → 7)
class CLIPMLPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super().__init__()
        self.clip = clip_model
        self.mlp = nn.Sequential(
            nn.Linear(self.clip.visual.output_dim, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        with torch.no_grad():
            feats = self.clip.encode_image(x)
        return self.mlp(feats)


def get_loader(domains, root, batch_size=32, shuffle=False):
    dataset = []
    for domain in domains:
        ds = datasets.ImageFolder(os.path.join(root, domain), transform=preprocess)
        dataset.extend(ds.samples)
    base_ds = datasets.ImageFolder(os.path.join(root, domains[0]), transform=preprocess)
    base_ds.samples = dataset
    loader = DataLoader(base_ds, batch_size=batch_size, shuffle=shuffle)
    return loader, len(base_ds.classes)


PACS_PATH = "/kaggle/input/pacs-dataset/kfold"
train_domains = ["cartoon", "photo", "sketch"]
test_domain = "art_painting"


train_loader, num_classes = get_loader(train_domains, root=PACS_PATH, shuffle=True)
test_loader, _ = get_loader([test_domain], root=PACS_PATH, shuffle=False)


model = CLIPMLPClassifier(clip_model, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.mlp.parameters(), lr=1e-4)

# Train + evaluate 
for epoch in range(20):
    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} [Training]"):
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(imgs)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"\nEpoch {epoch+1} Training Loss: {total_loss:.4f}")

    
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = 100 * correct / total
    print(f"Test Accuracy on '{test_domain}': {acc:.2f}%\n")
